# HI-SEAS Solar Insolation RNN Model

Author: Abhipray Sahoo
Date: 04/30/2017

### Goal: 
Estimate the solar radiation incident on the ground at HI-SEAS in order to predict power generated by solar panels. The model should be able to give best estimate for the solar radiation given meterological conditions. 

### Dataset:
A - NASA's HI-SEAS meterological data and solar irradiance. 

B - Additional climate data from Dark Sky API for the same dates as A

Uncomment cells to re-activate


## Load NASA dataset

In [3]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import seaborn as sns

DATA_PATH = '../hi-seas-data'

X_aug = pd.read_pickle('nasa_forecast_combined.pkl')

In [4]:
# Drop any nans 
X_aug = X_aug.dropna()
X_aug.describe()

,id,unix_secs,date,time,irradiance,speed,humidity,temperature,direction,pressure,humidity_fc,windSpeed_fc,windBearing_fc,precipIntensity_fc,precipProbability_fc,pressure_fc,visibility_fc,cloudCover_fc
count,32007,32007,32007,32007,32007.00,32007.00,32007.00,32007.0,32007.00,32007.0000,32007.00,32007.0,32007,32007.0,32007.0,32007.0,32007.0,32007.00
unique,32007,32007,120,8181,14111.00,37.00,94.00,38.0,17668.00,37.0000,68.00,848.0,356,173.0,69.0,779.0,200.0,67.00
top,33132,1478754303,2016-11-07,16:20:18,1.22,5.62,1.01,45.0,0.11,749.3146,0.82,2.6,153,0.0,0.0,1015.5,10.0,0.31
freq,1,1,288,24,2199.00,4567.00,1959.00,2801.0,93.00,4567.0000,1174.00,161.0,333,22866.0,22866.0,178.0,22626.0,6648.00


## Train small Fully Connected Network

1. Per feature scaling between min max
2. Train a fully connected network

In [5]:
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

tmp = X_aug.dropna()
print(tmp.shape)
y = np.array(tmp['irradiance'])
X = np.array(tmp[['temperature', 'direction', 'cloudCover_fc']]).astype(np.float)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=9)

(32007, 18)


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X.shape[1], )))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
model.fit(X_scaled, y, validation_split=0.1, shuffle=True, batch_size=32, epochs=60)

Train on 28806 samples, validate on 3201 samples
Epoch 1/60
28806/28806 [==============================] - 2s - loss: 115702.9942 - mean_squared_error: 115702.9942 - val_loss: 44665.4214 - val_mean_squared_error: 44665.4214
Epoch 2/60
28806/28806 [==============================] - 2s - loss: 79251.9111 - mean_squared_error: 79251.9111 - val_loss: 36804.1749 - val_mean_squared_error: 36804.1749
Epoch 3/60
28806/28806 [==============================] - 2s - loss: 61489.8532 - mean_squared_error: 61489.8532 - val_loss: 40548.5722 - val_mean_squared_error: 40548.5722
Epoch 4/60
28806/28806 [==============================] - 2s - loss: 53773.3669 - mean_squared_error: 53773.3669 - val_loss: 43843.6928 - val_mean_squared_error: 43843.6928
Epoch 5/60
28806/28806 [==============================] - 2s - loss: 50549.6240 - mean_squared_error: 50549.6240 - val_loss: 44633.8233 - val_mean_squared_error: 44633.8233
Epoch 6/60
28806/28806 [==============================] - 2s - loss: 49225.9818 - me

In [43]:
model.evaluate(scX_test, y_test)

31584/32007 [============================>.] - ETA: 0s

[2.5330733308598217, 2.5330733308598217]

In [44]:
model.predict(X_train[0][None,:])

array([[ 1.35]], dtype=float32)

In [45]:
y[0]

0.23901690047049992

In [34]:
# Five random example predictions
for i in range(5):
    idx = np.random.randint(X_test_pca.shape[0])
    ex_pred = svr.predict(X_test_pca[idx][None, :])[0]
    expected = y_test[idx]
    print("[{}] [{}] Predicted: {} Expected: {}".format(i, idx, ex_pred, expected))

[0] [7866] Predicted: 2.5801687986805852 Expected: 854.21
[1] [5127] Predicted: 2.5799998945207596 Expected: 3.26
[2] [4771] Predicted: 2.57999992394791 Expected: 1.23
[3] [211] Predicted: 2.580104353119218 Expected: 355.71
[4] [3257] Predicted: 2.5812482162468813 Expected: 8.09
